In [1]:
import os
from pathlib import Path
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

# --- GeoDataFrames si está disponible (reproyección), si no: fiona (sin reproyección) ---
try:
    import geopandas as gpd
    _HAVE_GPD = True
except Exception:
    import fiona
    _HAVE_GPD = False


# 1) Buscar escenas ya procesadas (donde existen outputs raster)
def buscar_escenas_con_outputs(root_rasters: Path):
    """
    Recorre recursivamente root_rasters y devuelve una lista de dicts:
    {"carpeta": Path, "ICU_intensity": Path|None, "ICU_mask": Path|None, "DeltaT": Path|None}
    """
    root_rasters = Path(root_rasters)
    escenas = []
    for r, d, files in os.walk(root_rasters):
        files_set_lower = {f.lower() for f in files}
        if "icu_intensity.tif" in files_set_lower:
            folder = Path(r)
            def _p(name):
                for f in files:
                    if f.lower() == name.lower():
                        return folder / f
                return None
            escenas.append({
                "carpeta": folder,
                "ICU_intensity": _p("ICU_intensity.TIF"),
                "ICU_mask": _p("ICU_mask.TIF"),
                "DeltaT": _p("DeltaT.TIF"),
            })
    return escenas


# 2) Indexar GPKG de límites en otra carpeta
def indexar_gpkgs(gpkg_root: Path):
    """
    Recorre recursivamente gpkg_root y devuelve lista de todos los .gpkg que contengan 'limite' en el nombre.
    """
    gpkg_root = Path(gpkg_root)
    lista = []
    for r, d, files in os.walk(gpkg_root):
        for f in files:
            if f.lower().endswith(".gpkg") and "limite" in f.lower():
                lista.append(Path(r) / f)
    return lista


# 3) Elegir el GPKG por escena (por nombre de ciudad = carpeta padre de la escena)
def elegir_gpkg_para_escena(scene_folder: Path, gpkg_paths: list[Path]):
    """
    Intenta emparejar por nombre de ciudad (carpeta padre de la escena).
    Si hay 1 solo gpkg en total, usa ese.
    Si hay varios y no matchea, lanza error (mejor pedir override).
    """
    city = scene_folder.parent.name.lower()  # p.ej. cartagena / barranquilla
    if not gpkg_paths:
        raise FileNotFoundError("No se encontraron GPKG de límite en la carpeta indicada.")

    # 1) Coincidencia por nombre de ciudad en el path del GPKG
    candidatos = [g for g in gpkg_paths if city in str(g.parent).lower() or city in g.name.lower()]
    if len(candidatos) == 1:
        return candidatos[0]
    if len(candidatos) > 1:
        # Si hay varios, elige el de carpeta más cercana (path más corto) o el más recientemente modificado
        candidatos.sort(key=lambda p: (len(str(p).split(os.sep)), -p.stat().st_mtime))
        return candidatos[0]

    # 2) Si no hubo match y solo hay 1 gpkg total, usa ese
    if len(gpkg_paths) == 1:
        return gpkg_paths[0]

    # 3) No se pudo decidir
    raise ValueError(f"No se pudo elegir un GPKG para la escena '{scene_folder.name}'. "
                     f"Provee gpkg_override o acomoda nombres/carpeta por ciudad.")


# Helper: leer geometrías del GPKG en el CRS del raster
def _geoms_en_crs(gpkg_path: Path, dst_crs):
    if _HAVE_GPD:
        gdf = gpd.read_file(gpkg_path)
        if gdf.empty:
            raise ValueError(f"El GPKG {gpkg_path} no tiene geometrías.")
        gdf = gdf.to_crs(dst_crs)
        geom_union = gdf.unary_union
        if geom_union.is_empty:
            raise ValueError(f"Geometría vacía tras reproyección: {gpkg_path}")
        if geom_union.geom_type == "MultiPolygon":
            return [mapping(g) for g in geom_union.geoms]
        return [mapping(geom_union)]
    else:
        # Fallback: se asume que el CRS ya coincide (no reproyecta)
        with fiona.open(gpkg_path) as src:
            if src.crs is None:
                raise ValueError(f"El GPKG {gpkg_path} no tiene CRS definido.")
            return [feat["geometry"] for feat in src]


# 4) Clip genérico de un raster contra un límite (lista de shapes)
def _clip_raster(in_tif: Path, shapes, out_tif: Path, nodata_default=None, compress="lzw"):
    in_tif = Path(in_tif)
    out_tif = Path(out_tif)
    out_tif.parent.mkdir(parents=True, exist_ok=True)

    with rasterio.open(in_tif) as src:
        try:
            out_img, out_transform = mask(
                src, shapes, crop=True, nodata=(src.nodata if src.nodata is not None else nodata_default)
            )
        except ValueError as e:
            print(f"ℹ️ Clip omitido ({in_tif.name}): {e}")
            return None

        meta = src.meta.copy()
        meta.update({
            "height": out_img.shape[1],
            "width":  out_img.shape[2],
            "transform": out_transform,
            "compress": compress
        })
        if nodata_default is not None and (meta.get("nodata") is None):
            meta.update({"nodata": nodata_default})

    with rasterio.open(out_tif, "w", **meta) as dst:
        dst.write(out_img)
    print(f"✂️  {out_tif.name} escrito en {out_tif.parent}")
    return out_tif


# 5) Clip de outputs de una escena (paths pueden estar en otra raíz)
def clip_outputs_de_escena(scene: dict, gpkg_path: Path, out_dir: Path | None = None):
    """
    scene: {"carpeta": Path, "ICU_intensity": Path|None, "ICU_mask": Path|None, "DeltaT": Path|None}
    gpkg_path: GPKG del límite (puede estar en otra ruta).
    out_dir: carpeta destino para los *_clip.TIF. Por defecto, la carpeta de la escena.
    """
    carpeta = Path(scene["carpeta"])
    out_dir = Path(out_dir) if out_dir else carpeta      # <-- guardar en la carpeta de la escena
    out_dir.mkdir(parents=True, exist_ok=True)

    resultados = {}

    def _clip_one(in_path: Path, out_name: str, nodata_default):
        if in_path is None or not Path(in_path).exists():
            return None
        with rasterio.open(in_path) as src:
            shapes = _geoms_en_crs(gpkg_path, src.crs)

            out_img, out_transform = mask(
                src, shapes, crop=True,
                nodata=(src.nodata if src.nodata is not None else nodata_default)
            )
            meta = src.meta.copy()
            meta.update({
                "height": out_img.shape[1],
                "width":  out_img.shape[2],
                "transform": out_transform,
                "compress": "lzw"
            })
            if nodata_default is not None and (meta.get("nodata") is None):
                meta.update({"nodata": nodata_default})

        out_path = out_dir / out_name                    # <-- aquí cambia el destino
        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(out_img)
        print(f"✂️  {out_name} escrito en {out_path}")
        return out_path

    if scene.get("DeltaT"):
        resultados["DeltaT_clip"] = str(_clip_one(scene["DeltaT"], "DeltaT_clip.TIF", nodata_default=np.nan) or "")
    if scene.get("ICU_mask"):
        resultados["ICU_mask_clip"] = str(_clip_one(scene["ICU_mask"], "ICU_mask_clip.TIF", nodata_default=0) or "")
    if scene.get("ICU_intensity"):
        resultados["ICU_intensity_clip"] = str(_clip_one(scene["ICU_intensity"], "ICU_intensity_clip.TIF", nodata_default=0) or "")

    return resultados



# 6) Orquestador: raíz de ráster distinta a raíz de GPKG
def clip_lote(root_rasters: Path, gpkg_root: Path, gpkg_override: Path | None = None):
    """
    - root_rasters: raíz donde están todas las escenas con ICU_intensity/ICU_mask/DeltaT.
    - gpkg_root: raíz donde están los GPKG (pueden estar en subcarpetas por ciudad).
    - gpkg_override: si lo pasas, usa ese mismo GPKG para todas las escenas.
    """
    escenas = buscar_escenas_con_outputs(root_rasters)
    print(f"🔎 Escenas con outputs encontradas: {len(escenas)}")
    if not escenas:
        return []

    if gpkg_override:
        gpkg_paths = [Path(gpkg_override)]
    else:
        gpkg_paths = indexar_gpkgs(gpkg_root)

    resultados_global = []
    for sc in escenas:
        try:
            gpkg = Path(gpkg_override) if gpkg_override else elegir_gpkg_para_escena(sc["carpeta"], gpkg_paths)
            res = clip_outputs_de_escena(sc, gpkg)
            resultados_global.append({"carpeta": str(sc["carpeta"]), "gpkg": str(gpkg), **res})
        except Exception as e:
            print(f"⚠️  Error clip en {sc['carpeta'].name}: {e}")
            resultados_global.append({"carpeta": str(sc["carpeta"]), "gpkg": None, "error": str(e)})
    return resultados_global


In [2]:
root_rasters = Path("/home/rohi/Downloads/maestria/10-heat_island/barranquilla")     # donde están las escenas con outputs
gpkg_root    = Path("/home/rohi/Downloads/maestria/limites/barranquilla")     # donde están los .gpkg (en subcarpetas por ciudad, por ej.)

# A) Emparejar por ciudad automáticamente
resultados = clip_lote(root_rasters, gpkg_root)
# resultados es una lista de dicts con rutas *_clip.TIF o errores

# B) Usar SIEMPRE el mismo GPKG para todas las escenas (override)
# resultados = clip_lote(root_rasters, gpkg_root, gpkg_override=Path("/ruta/a/LIMITES/limite.gpkg"))


🔎 Escenas con outputs encontradas: 10


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20210212_20220526_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20210212_20220526_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20210212_20220526_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20160114_20200907_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20160114_20200907_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20160114_20200907_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20230117_20230131_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20230117_20230131_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20230117_20230131_02_T1/ICU_intensity_clip.TIF


...

/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC09_L2SP_009052_20220223_20230427_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC09_L2SP_009052_20220223_20230427_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC09_L2SP_009052_20220223_20230427_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20190106_20200830_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20190106_20200830_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20190106_20200830_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20200329_20200822_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20200329_20200822_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20200329_20200822_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquilla/LC08_L2SP_009052_20180204_20200902_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/barranquil

/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


In [3]:
root_rasters = Path("/home/rohi/Downloads/maestria/10-heat_island/cartagena")     # donde están las escenas con outputs
gpkg_root    = Path("/home/rohi/Downloads/maestria/limites/cartagena")     # donde están los .gpkg (en subcarpetas por ciudad, por ej.)

# A) Emparejar por ciudad automáticamente
resultados = clip_lote(root_rasters, gpkg_root)
# resultados es una lista de dicts con rutas *_clip.TIF o errores

# B) Usar SIEMPRE el mismo GPKG para todas las escenas (override)
# resultados = clip_lote(root_rasters, gpkg_root, gpkg_override=Path("/ruta/a/LIMITES/limite.gpkg"))

🔎 Escenas con outputs encontradas: 10
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20200329_20200822_02_T1/ICU_intensity_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20240120_20240129_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20240120_20240129_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20240120_20240129_02_T1/ICU_intensity_clip.TIF


...

/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_010053_20220222_20220302_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_010053_20220222_20220302_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_010053_20220222_20220302_02_T1/ICU_intensity_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20210212_20220526_02_T1/DeltaT_clip.TIF
✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20210212_20220526_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/cartagena/LC08_L2SP_009053_20210212_20220526_02_T1/ICU_intensity_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


In [4]:
root_rasters = Path("/home/rohi/Downloads/maestria/10-heat_island/santa_marta")     # donde están las escenas con outputs
gpkg_root    = Path("/home/rohi/Downloads/maestria/limites/santa_marta")     # donde están los .gpkg (en subcarpetas por ciudad, por ej.)

# A) Emparejar por ciudad automáticamente
resultados = clip_lote(root_rasters, gpkg_root)
# resultados es una lista de dicts con rutas *_clip.TIF o errores

# B) Usar SIEMPRE el mismo GPKG para todas las escenas (override)
# resultados = clip_lote(root_rasters, gpkg_root, gpkg_override=Path("/ruta/a/LIMITES/limite.gpkg"))


🔎 Escenas con outputs encontradas: 10
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20210212_20220526_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20210212_20220526_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20210212_20220526_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20160114_20200907_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20160114_20200907_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20160114_20200907_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20230117_20230131_02_T1/DeltaT_clip.TIF


...

/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20200329_20200822_02_T1/ICU_intensity_clip.TIF
✂️  DeltaT_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/DeltaT_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union


✂️  ICU_mask_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/ICU_mask_clip.TIF
✂️  ICU_intensity_clip.TIF escrito en /home/rohi/Downloads/maestria/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/ICU_intensity_clip.TIF


/tmp/ipykernel_325139/3622724031.py:93: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = gdf.unary_union
